In [40]:
import numpy as np
import pandas as pd
import requests
import json
import os
import ta

pd.set_option('display.max_columns', 100)


In [41]:
EXCHANGE_ID = "ec6ab160-fcb7-44ff-9ae8-a548c92dad06"


BTCUSDT_DAY_ID = "9c1d60c0-5562-452e-9b74-88495d76e0c3"
# BTCUSDT_HOUR_ID = "7aa396ec-8c6a-44fd-89f2-12e7f998f883"
# BTCUSDT_MINUTE_ID = "31deee63-e4bb-45ad-b020-8759b5c69e0f"
# ETHUSDT_DAY_ID = "67da12e9-6fd9-4401-b16a-1c2dd589ff01"
# ETHUSDT_HOUR_ID = "35515212-5467-45b4-b2e5-3ae048e482e7"
# ETHUSDT_MINUTE_ID = "38d4f96b-26b7-420a-a34f-912f5ad23eaf"


ASSET_ID = BTCUSDT_DAY_ID

# base_url = "http://172.24.100.128:5000/"
base_url = "http://localhost:5000/"


PROXIES = {
    
   'http': 'http://discproxy.virtual.uniandes.edu.co:443',
}


In [42]:
# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_prices_between_unix_time(base_url, asset_id, unix_time_start, unix_time_end):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "assets/" + asset_id + "/indicators_unix_between/",
                            params={'unix_time_start': unix_time_start, 'unix_time_end': unix_time_end})#,proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # print("get prices between status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        # drop columns that are not needed
        df.drop(columns=['updated_at', 'created_at', 'date_time_gmt_5','ignore','asset_id'], inplace=True)

        # Return the DataFrame containing the retrieved price data
        return df

    else:
        # print("Failed api_request_get_prices_between_unix_time")
        print("get prices between status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    


def api_request_get_asset_from_asset_id(base_url, exchange_id, asset_id):
    # try:
    # Make GET request to API endpoint

    response = requests.get(base_url + "exchanges/" +
                            exchange_id + "/asset/"+asset_id)#,proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        print("get asset status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)
        # Return the Unix timestamp of the last recorded price for the specified asset ID

        print("symbol: " + df["symbol"].iloc[0])
        print("interval: " + df["interval"].iloc[0])
        return df
    else:
        print(response.content)
        json_data = json.loads(response.content)
        df = pd.json_normalize(json_data)
        return df



# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_all_indicators_from_price(base_url, price_id,):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "prices/" + price_id + "/indicators/")#,proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        if df.empty:
            return df
        else:
        
            df.drop(columns=['id', 'created_at', 'updated_at','date_time_utc','unix_time','date_time_gmt_5'], inplace=True)

            df.set_index('price_id', inplace=True)

            pivoted_df = pd.pivot_table(df, index='price_id', columns='name', values='value', aggfunc=sum, fill_value=0)

            pivoted_df = pivoted_df.replace(0, np.nan)

            return pivoted_df

    else:
        print("get indicators status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    

def get_indicators_and_merge(base_url,exchange_id, asset_id, unix_time_start, unix_time_end):


    # Get the asset from the API
    asset = api_request_get_asset_from_asset_id(base_url,exchange_id,asset_id)

    interval = asset["interval"].iloc[0]
    
    if interval == "minute":
        first_indicator_time = unix_time_start - 6000

    if interval == "hour":
        first_indicator_time = unix_time_start - 360000

    if interval == "day":
        first_indicator_time = unix_time_start - 8640000


    df = api_request_get_prices_between_unix_time(base_url, asset_id, first_indicator_time, unix_time_end)
    
    df_prices_wit_indicators_all = add_indicators(df)

    df_prices_wit_indicators = df_prices_wit_indicators_all.loc[df_prices_wit_indicators_all['unix_time'] >= unix_time_start]

    df_assets = pd.DataFrame()

    for i in df['id']:

        df2 = api_request_get_all_indicators_from_price(base_url, i)

        if not df2.empty:
            df_assets = pd.concat([df_assets, df2])
        
    if not df_assets.empty:
        df_merge = pd.merge(df_prices_wit_indicators, df_assets, left_on='id', right_on='price_id', how='left')
    else:
        df_merge = df_prices_wit_indicators

    return df_merge


# method to add the thecnic indicators to the dataframe
def add_indicators(df):

    # momentum
    df['ao'] = ta.momentum.awesome_oscillator(
        high=df['high_price'], low=df['low_price'])
    df['kama'] = ta.momentum.kama(close=df['close_price'])
    df['ppo'] = ta.momentum.ppo(close=df['close_price'])
    df['pvo'] = ta.momentum.pvo(volume=df['volume'])
    df['roc'] = ta.momentum.roc(close=df['close_price'])
    df['rsi'] = ta.momentum.rsi(close=df['close_price'])
    df['stochrsi'] = ta.momentum.stochrsi(close=df['close_price'])
    df['stoch'] = ta.momentum.stoch(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['tsi'] = ta.momentum.tsi(close=df['close_price'])
    df['uo'] = ta.momentum.ultimate_oscillator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['wri'] = ta.momentum.williams_r(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # volume
    df['accdist'] = ta.volume.acc_dist_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['cmf'] = ta.volume.chaikin_money_flow(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['emv'] = ta.volume.ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['fi'] = ta.volume.force_index(
        close=df['close_price'], volume=df['volume'])
    df['mfi'] = ta.volume.money_flow_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['nvi'] = ta.volume.negative_volume_index(
        close=df['close_price'], volume=df['volume'])
    df['obv'] = ta.volume.on_balance_volume(
        close=df['close_price'], volume=df['volume'])
    df['smaemv'] = ta.volume.sma_ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['vpt'] = ta.volume.volume_price_trend(
        close=df['close_price'], volume=df['volume'])
    df['vwap'] = ta.volume.volume_weighted_average_price(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])

    # volatility
    df['atr'] = ta.volatility.average_true_range(
        high=df['high_price'], low=df['low_price'], close=df['close_price'])
    df['ulcer'] = ta.volatility.ulcer_index(close=df['close_price'])

    df['bbh'] = ta.volatility.bollinger_hband(close=df['close_price'])
    df['bbl'] = ta.volatility.bollinger_lband(close=df['close_price'])
    df['bbhi'] = ta.volatility.bollinger_hband_indicator(
        close=df['close_price'])
    df['bbli'] = ta.volatility.bollinger_lband_indicator(
        close=df['close_price'])
    df['bbmavg'] = ta.volatility.bollinger_mavg(close=df['close_price'])
    df['bb_pb'] = ta.volatility.bollinger_pband(close=df['close_price'])
    df['bb_wb'] = ta.volatility.bollinger_wband(close=df['close_price'])

    df['dchb'] = ta.volatility.donchian_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dclb'] = ta.volatility.donchian_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcmb'] = ta.volatility.donchian_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcpb'] = ta.volatility.donchian_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcwb'] = ta.volatility.donchian_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['kchb'] = ta.volatility.keltner_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclb'] = ta.volatility.keltner_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kchbi'] = ta.volatility.keltner_channel_hband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclbi'] = ta.volatility.keltner_channel_lband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcmb'] = ta.volatility.keltner_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcpb'] = ta.volatility.keltner_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcwb'] = ta.volatility.keltner_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # trend

    # En dos de estos tres hay algo que esta botando warning
    df['adx'] = ta.trend.adx(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['adx_neg'] = ta.trend.adx_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['adx_pos'] = ta.trend.adx_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['aroon_up'] = ta.trend.aroon_up(close=df['close_price'])
    df['aroon_down'] = ta.trend.aroon_down(close=df['close_price'])

    df['cci'] = ta.trend.cci(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['dpo'] = ta.trend.dpo(close=df['close_price'])
    df['kst'] = ta.trend.kst(close=df['close_price'])
    df['kst_sig'] = ta.trend.kst_sig(close=df['close_price'])
    df['ema'] = ta.trend.ema_indicator(close=df['close_price'])

    df['ichimoku_a'] = ta.trend.ichimoku_a(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_b'] = ta.trend.ichimoku_b(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_base_line'] = ta.trend.ichimoku_base_line(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_conversion_line'] = ta.trend.ichimoku_conversion_line(
        high=df['high_price'], low=df['low_price'])

    df['macd'] = ta.trend.macd(close=df['close_price'])
    df['macd_diff'] = ta.trend.macd_diff(close=df['close_price'])
    df['macd_signal'] = ta.trend.macd_signal(close=df['close_price'])

    df['mi'] = ta.trend.mass_index(high=df['high_price'], low=df['low_price'])
    df['sma'] = ta.trend.sma_indicator(close=df['close_price'])
    df['wma'] = ta.trend.wma_indicator(close=df['close_price'])
    df['stc'] = ta.trend.stc(close=df['close_price'])
    df['trix'] = ta.trend.trix(close=df['close_price'])

    df['vi_pos'] = ta.trend.vortex_indicator_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['vi_neg'] = ta.trend.vortex_indicator_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    return df

In [46]:
df_prueba = get_indicators_and_merge(base_url, EXCHANGE_ID, ASSET_ID, 1680134400, 1682278312)
df_prueba.sort_values(by='unix_time', ascending=True, inplace=True)
df_prueba.tail(15)

get asset status code: 200
symbol: BTCUSDT
interval: day


/Users/julianoliverosforero/Desktop/2023-1/Tesis/cryptocurrency-market-prediction-project/data-loader/env/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/julianoliverosforero/Desktop/2023-1/Tesis/cryptocurrency-market-prediction-project/data-loader/env/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


,high_price,unix_time,volume,date_time_utc,qav,num_trades,open_price,taker_base_vol,close_price,taker_quote_vol,low_price,id,ao,kama,ppo,pvo,roc,rsi,stochrsi,stoch,tsi,uo,wri,accdist,cmf,emv,fi,mfi,nvi,obv,smaemv,vpt,vwap,atr,ulcer,bbh,bbl,bbhi,bbli,bbmavg,bb_pb,bb_wb,dchb,dclb,dcmb,dcpb,dcwb,kchb,kclb,kchbi,kclbi,kcmb,kcpb,kcwb,adx,adx_neg,adx_pos,aroon_up,aroon_down,cci,dpo,kst,kst_sig,ema,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,macd,macd_diff,macd_signal,mi,sma,wma,stc,trix,vi_pos,vi_neg,NaturalGas,Russell2000,cocoa,coffee,corn,cotton,dowJones,gold,nasdaq,oat,rice,silver,soybean,spy500,sugar,usBond,usBond10y,usBond2y,usBond5y,vwheat
12,28100.00,1680825600,24762.09387,2023-04-07T00:00:00+00:00,6.913237e+08,655634,28033.83,11983.13367,27906.33,3.345595e+08,27766.94,7e9ccd80-ba9e-4357-b23f-8d580276f456,4540.553353,25352.823701,6.307090,-35.838119,13.116604,53.870710,0.040908,55.953089,16.509712,50.206901,-44.046911,1.284638e+06,-0.117453,-3.230147e+09,2.895304e+07,55.129826,1624.398156,2.769287e+06,2.002107e+08,-2100.468467,27186.140773,2002.903406,4.886533,31811.264418,21999.767582,0.0,0.0,26905.5160,0.602004,36.466488,31000.00,21580.00,28754.070,0.311272,15.657076,28116.310000,25750.216000,0.0,0.0,26933.263000,0.911255,8.785025,14.602608,31.833036,33.995494,92.0,24.0,31.823246,811.4940,270.248764,241.794080,28223.703523,27533.6425,23666.500,26266.385,28800.900,1674.483271,64.163958,1610.319313,27.461957,28564.914167,29064.434444,49.999997,0.984592,0.909454,0.711224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28154.99,1680912000,19479.96735,2023-04-08T00:00:00+00:00,5.453245e+08,624159,27906.34,9729.24214,27938.38,2.723667e+08,27859.02,9d985c77-fe5e-4e0d-8fc5-913a0f43f8ee,4859.426147,24074.872783,7.233758,-12.975806,28.310914,65.016633,0.907076,92.113070,18.181141,73.524237,-7.886930,8.542685e+05,0.091307,3.052765e+08,1.029340e+08,42.168185,1504.082229,1.207282e+06,2.012706e+08,-888.465230,22682.636342,1740.743350,5.513606,28306.975113,17939.670887,0.0,0.0,23123.3230,0.964446,44.834837,28656.69,17892.05,24102.890,0.921131,37.148603,24012.133667,22154.998667,1.0,0.0,23083.566167,3.114142,8.045269,23.017992,25.585325,49.671669,92.0,12.0,161.725095,-1498.1330,210.136096,133.606288,24803.478942,23355.5800,22684.205,22608.270,24102.890,1673.189123,713.584819,959.604303,25.351944,24082.486667,25927.491778,NaN,0.603257,0.957376,0.877934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28530.00,1680998400,32531.16101,2023-04-09T00:00:00+00:00,9.139159e+08,777589,27938.38,16885.70757,28323.76,4.744543e+08,27800.00,53089623-6f54-4abf-b2fb-db47378fe38f,5837.357588,24702.732203,8.042685,-12.040215,27.596230,65.861922,0.948867,96.344482,21.706918,71.596609,-3.655518,1.042392e+06,0.059023,6.601294e+08,8.525169e+07,44.536266,1515.764159,1.660743e+06,2.719723e+08,2770.101925,23474.513324,1628.400388,5.072955,29427.689455,18470.058545,0.0,0.0,23948.8740,0.899255,45.754263,28656.69,19549.09,24102.890,0.963445,35.770382,24846.222667,22978.322667,1.0,0.0,23912.272667,2.861736,7.811470,23.710245,25.846970,46.186912,84.0,4.0,119.613236,-431.1540,229.989031,169.599168,25774.905633,23535.9575,22684.205,22608.270,24463.645,1918.681110,630.773193,1287.907917,26.023111,25132.993333,27259.158000,NaN,0.823225,0.996102,0.784826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,29770.00,1681084800,67754.06220,2023-04-10T00:00:00+00:00,1.955301e+09,1285695,28323.76,36147.62326,29637.34,1.043139e+09,28170.00,45804ca0-dc57-4d6c-8d9c-b4cb3dd76a1a,4693.420559,23789.760971,6.520456,-11.075159,23.797378,61.981427,0.945932,98.380220,15.454626,53.883062,-1.619780,1.345000e+06,-0.099322,2.916607e+09,6.746199e+07,51.951971,1749.105740,2.836110e+06,2.987097e+08,1507.150911,25322.865860,2105.698556,4.766836,29693.967658,20533.517342,0.0,0.0,25113.7425,0.993818,36.475847,29770.00,21532.77,26414.590,0.980232,25.023017,26299.165500,23978.503500,